In [1]:
# --- Imports ---
import os
import random
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import classification_report, accuracy_score
import re

In [2]:
# --- Configuration ---
MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"

CSV_PATH = r"C:\Users\stdFurqan\Downloads\arabic_ham_SPAM\test.csv"
TEXT_COL = "Cleaned Text"
LABEL_COL = "Label"
LABELS = ["Ham", "Spam"]
SEED = 20
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- Reproducibility ---
def set_seed(seed=20):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(SEED)

In [3]:
# ✅ Local path where model is stored
MODEL_PATH = r"C:\Users\stdFurqan\Downloads\lama_models_download\LAMA_3.2(1b)"

print(f"🧠 Loading model from {MODEL_PATH} ...")

# ✅ Load tokenizer from local folder
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# ✅ Load model from local folder
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,   # ✅ best for 40-series
    device_map="auto"
)

model.config.pad_token_id = tokenizer.pad_token_id
model.eval()

print("✅ Model loaded successfully from local directory!")

🧠 Loading model from C:\Users\stdFurqan\Downloads\lama_models_download\LAMA_3.2(1b) ...


`torch_dtype` is deprecated! Use `dtype` instead!
The module name LAMA_3_dot_2(1b) (originally LAMA_3.2(1b)) is not a valid Python identifier. Please rename the original module to avoid import issues.


✅ Model loaded successfully from local directory!


In [4]:
 
# # Step 1: Training data (4-shot) ha,m and spam
# train_dataset = [

#     InputExample(guid=0, text_a="خبراء مختصون بالصحه العامه يؤكدون ان علي الناس ارتداء الاقنعه الواقيه والالتزام بالتباعد الاجتماعي حتي بعد تلقي القاح ضد كورونا العربيه", label=0),
#     InputExample(guid=1, text_a="مؤكدا تركيب نحو جهاز طرد مركزي متطور في منشاتي نطنز وفوردو الحكومه الايرانيه التراجع عن اعاده فرض العقوبات الاميه جيد لكن لا يكفي ايران العربيه", label=0),
#     InputExample(guid=2, text_a="بعد ان حكم عليه بالسجن سنوات بتهمه المساس بالوحده الوطنيه الحدث تواكب وصول المفرج عنه الصحفي خالد درارني الي منزله في الجزائر غداه اعلان العفو الرئاسي", label=0),
#     InputExample(guid=3, text_a="الصحه تجري فحصا ضمن خطها لتوسيع نطاق الفحوصات وتكشف عن اصابات جديده بفيروس كورونا المستجد و حاله شفاء و حاله وفاه خلال الساعات ال الماضيه", label=0),

#     InputExample(guid=4, text_a="النجاح يبدا مع شهاده الايلتس شهاده الايلتس معتمده كاثبات لكفاءه الغه الانجليزيه لغايات التعليم العالي والهجرات العالميه تواصل معنا الان واحصل علي ايلتس موثقه ومعتمده من قبل المعهد البريطاني الثقافي في خلال ايام التواصل عبر الخاص الهلال الرايد", label=1),
#     InputExample(guid=5, text_a="تخلص من التدخين باقصي سرعه و حافظ ع صحتك و صحه عائلتك حارب الحلول المؤقته مع مجموعه الاقلاع عن التدخين من مكونات الطبيعه لتواصل خاص", label=1),
#     InputExample(guid=6, text_a="البديل الامن والفعال لفياجرا والسنافي الماالتي ماكا اقوي علاج ل سرعه القذف قله الرغبه الجنسيه ضعف الانتصاب مشاكل البروستاتا عشبي امن لمرضي السكر والضغط لطلب حياكم التواصل واتساب رابط واتساب", label=1),
#     InputExample(guid=7, text_a="وزاره التجاره  تحذر من التجاوب او مجاراه اعلانات عبر الانترنت تستغل فيها صور بعض المشاهير لجذب العملاء لغرض التداول با", label=1),
# ]


In [5]:


# --- Load dataset ---
df = pd.read_csv(CSV_PATH)
texts = df[TEXT_COL].astype(str).tolist()
true_labels = df[LABEL_COL].astype(str).tolist()

PROMPT_TEMPLATE = (
    "You are an intelligent Arabic language assistant.\n"
    "Your task is to classify Arabic tweets as either Ham (normal/non-spam news or message) "
    "or Spam (unsolicited, promotional, or advertisement content).\n"
    "Use the examples below as guidance.\n\n"
    
    "=== Ham Examples ===\n"
    "1 Text: خبراء مختصون بالصحه العامه يؤكدون ان علي الناس ارتداء الاقنعه الواقيه والالتزام بالتباعد الاجتماعي حتي بعد تلقي القاح ضد كورونا العربيه\n   Label: Ham\n"
    "2 Text: مؤكدا تركيب نحو جهاز طرد مركزي متطور في منشاتي نطنز وفوردو الحكومه الايرانيه التراجع عن اعاده فرض العقوبات الاميه جيد لكن لا يكفي ايران العربيه\n   Label: Ham\n"
    "3 Text: بعد ان حكم عليه بالسجن سنوات بتهمه المساس بالوحده الوطنيه الحدث تواكب وصول المفرج عنه الصحفي خالد درارني الي منزله في الجزائر غداه اعلان العفو الرئاسي\n   Label: Ham\n"
    "4 Text: الصحه تجري فحصا ضمن خطها لتوسيع نطاق الفحوصات وتكشف عن اصابات جديده بفيروس كورونا المستجد و حاله شفاء و حاله وفاه خلال الساعات ال الماضيه\n   Label: Ham\n\n"

    "=== Spam Examples ===\n"
    "5 Text: النجاح يبدا مع شهاده الايلتس شهاده الايلتس معتمده كاثبات لكفاءه الغه الانجليزيه لغايات التعليم العالي والهجرات العالميه تواصل معنا الان واحصل علي ايلتس موثقه ومعتمده من قبل المعهد البريطاني الثقافي في خلال ايام التواصل عبر الخاص الهلال الرايد\n   Label: Spam\n"
    "6 Text: تخلص من التدخين باقصي سرعه و حافظ ع صحتك و صحه عائلتك حارب الحلول المؤقته مع مجموعه الاقلاع عن التدخين من مكونات الطبيعه لتواصل خاص\n   Label: Spam\n"
    "7 Text: البديل الامن والفعال لفياجرا والسنافي الماالتي ماكا اقوي علاج ل سرعه القذف قله الرغبه الجنسيه ضعف الانتصاب مشاكل البروستاتا عشبي امن لمرضي السكر والضغط لطلب حياكم التواصل واتساب رابط واتساب\n   Label: Spam\n"
    "8 Text: وزاره التجاره  تحذر من التجاوب او مجاراه اعلانات عبر الانترنت تستغل فيها صور بعض المشاهير لجذب العملاء لغرض التداول با\n   Label: Spam\n\n"
         
    "Reply with only one word: Ham, or Spam.\n"
    "Now classify the following text in the same way.\n\n"
    "Text: {text}\n\n"
    "Label:"
)


pattern = re.compile(r"\b(Ham|Spam)\b", re.IGNORECASE)

def extract_label(output):
    m = pattern.search(output)
    return m.group(1).capitalize() if m else "Unknown"

pred_labels = []
batch_size = 4  # smaller batch to fit in T4 VRAM
for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    prompts = [PROMPT_TEMPLATE.format(text=t) for t in batch]
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=6,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    for prompt, full_out in zip(prompts, decoded):
        gen = full_out[len(prompt):].strip() if full_out.startswith(prompt) else full_out
        pred_labels.append(extract_label(gen))

# --- Evaluation ---
report = classification_report(true_labels, pred_labels, labels=LABELS, digits=4, zero_division=0)
acc = accuracy_score(true_labels, pred_labels)
print("\n📊 Classification Report:\n")
print(report)
print(f"Accuracy: {acc:.4f}")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



📊 Classification Report:

              precision    recall  f1-score   support

         Ham     0.9030    0.7637    0.8276      3390
        Spam     0.2751    0.5223    0.3604       582

    accuracy                         0.7283      3972
   macro avg     0.5891    0.6430    0.5940      3972
weighted avg     0.8110    0.7283    0.7591      3972

Accuracy: 0.7283
